In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [16]:
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import os
import json
import csv
import sys
csv.field_size_limit(sys.maxsize)

class TranscriptDataset(Dataset):
    def __init__(self, dataset_path = '/content/'):
        super().__init__()

        path = os.path.join(dataset_path, 'podcastdata_dataset.csv')

        self.list = []
        self.end_of_text_token = "<|endoftext|>"

        with open(path) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')

            x = 0
            for row in csv_reader:
                str = f"text:{row[3]}{self.end_of_text_token}"
                self.list.append(str)

    def __len__(self):
        return len(self.list)

    def __getitem__(self, item):
        return self.list[item]


In [17]:
dataset = TranscriptDataset()
loader = DataLoader(dataset, batch_size=1, shuffle=True)


In [23]:
BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 3e-5
WARMUP_STEPS = 5000
MAX_SEQ_LEN = 400
from transformers import AdamW, get_linear_schedule_with_warmup

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [25]:
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps = -1)
proc_seq_count = 0
sum_loss = 0.0
batch_count = 0

tmp_transcripts_tens = None
models_folder = "trained_models"
"""if not os.path.exists(models_folder):
    os.mkdir(models_folder)"""
for epoch in range(EPOCHS):

    print(f"EPOCH {epoch} started" + '=' * 30)

    for idx,transcript in enumerate(loader):

        #################### "Fit as many transcript sequences into MAX_SEQ_LEN sequence as possible" logic start ####
        transcript_tens = torch.tensor(tokenizer.encode(transcript[0])).unsqueeze(0).to(device)
        #Skip sample from dataset if it is longer than MAX_SEQ_LEN
        if transcript_tens.size()[1] > MAX_SEQ_LEN:
            continue

        #The first transcript sequence in the sequence
        if not torch.is_tensor(tmp_transcripts_tens):
            tmp_transcripts_tens = transcript_tens
            continue
        else:
            #The next transcript does not fit in so we process the sequence and leave the last transcript
            #as the start for next sequence
            if tmp_transcripts_tens.size()[1] + transcript_tens.size()[1] > MAX_SEQ_LEN:
                work_transcripts_tens = tmp_transcripts_tens
                tmp_transcripts_tens = transcript_tens
            else:
                #Add the transcript to sequence, continue and try to add more
                tmp_transcripts_tens = torch.cat([tmp_transcripts_tens, transcript_tens[:,1:]], dim=1)
                continue
        ################## Sequence ready, process it trough the model ##################

        outputs = model(work_transcripts_tens, labels=work_transcripts_tens)
        loss, logits = outputs[:2]
        loss.backward()
        sum_loss = sum_loss + loss.detach().data

        proc_seq_count = proc_seq_count + 1
        if proc_seq_count == BATCH_SIZE:
            proc_seq_count = 0
            batch_count += 1
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            model.zero_grad()

        if batch_count == 100:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0

    # Store the model after each epoch to compare the performance of them
    #torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_medium_transcriptr_{epoch}.pt"))


EPOCH 0 started==============================


Token indices sequence length is longer than the specified maximum sequence length for this model (16656 > 1024). Running this sequence through the model will result in indexing errors


EPOCH 1 started==============================
EPOCH 2 started==============================
EPOCH 3 started==============================
EPOCH 4 started==============================


In [27]:
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [30]:
transcripts_output_file_path = '/content/generated_transcript.text'
model.eval()

transcript_num = 0
with torch.no_grad():

        for transcript_idx in range(1000):

            transcript_finished = False

            cur_ids = torch.tensor(tokenizer.encode("transcript:")).unsqueeze(0).to(device)

            for i in range(100):
                outputs = model(cur_ids, labels=cur_ids)
                loss, logits = outputs[:2]
                softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(from only one in this case) batch and the last predicted embedding
                if i < 3:
                    n = 20
                else:
                    n = 3
                next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n) #Randomly(from the topN probability distribution) select the next word
                cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word to the running sequence

                if next_token_id in tokenizer.encode('<|endoftext|>'):
                    transcript_finished = True
                    break


            if transcript_finished:

                transcript_num = transcript_num + 1

                output_list = list(cur_ids.squeeze().to('cpu').numpy())
                output_text = tokenizer.decode(output_list)

                with open(transcripts_output_file_path, 'a') as f:
                    f.write(f"{output_text} \n\n")